In [8]:
import cv2
import numpy as np
from scipy.spatial.transform import Rotation as R


def generate_pixel_to_world_map(K, D, R_wc, t_wc, image_size, plane_z=0.0):
    H, W = image_size
    xyz_map = np.zeros((H, W, 3), dtype=np.float32)

    # 像素坐标网格
    u, v = np.meshgrid(np.arange(W), np.arange(H))  # shape: (H, W)
    pixel_coords = np.stack([u, v], axis=-1).astype(np.float32)  # (H, W, 2)
    pixel_coords_flat = pixel_coords.reshape(-1, 1, 2)

    # 去畸变得到归一化相机坐标
    undistorted_pts = cv2.fisheye.undistortPoints(pixel_coords_flat, K, D)  # (N,1,2)
    undistorted_pts = undistorted_pts.reshape(-1, 2)

    # 单位射线
    rays_cam = np.concatenate([undistorted_pts, np.ones((undistorted_pts.shape[0], 1))], axis=1)

    # 相机坐标系 → 世界坐标系的射线方向
    rays_world = (R_wc @ rays_cam.T).T  # (N,3)

    # ✅ 正确计算相机中心
    cam_center = t_wc.flatten()  # shape (3,)

    # 与 z=plane_z 的平面求交
    t_vals = (plane_z - cam_center[2]) / rays_world[:, 2]
    pts_world = cam_center + rays_world * t_vals[:, np.newaxis]

    xyz_map = pts_world.reshape(H, W, 3).astype(np.float32)
    return xyz_map

def save_xyz_map_binary(xyz_map: np.ndarray, file_path: str):
    # 确保是 float32 类型，等价于 CV_32FC3
    if xyz_map.dtype != np.float32:
        xyz_map = xyz_map.astype(np.float32)
    
    # 写入二进制文件
    with open(file_path, 'wb') as f:
        f.write(xyz_map.tobytes())

def compute_world_projection_errors(img_points, obj_points, xyz_map):
    """
    计算像素反投影到世界坐标后，与原始3D目标点之间的欧式距离误差。

    参数:
    - img_points: (N, 2) 图像中的像素点，二维数组。
    - obj_points: (N, 3) 对应的世界坐标点，三维数组。
    - xyz_map: (H, W, 3) 每个像素对应的世界坐标反投影结果。

    返回:
    - dist_errors: 每个点的欧式距离误差（单位：米）
    - mean_error: 平均误差
    """
    reprojected_world_points = []

    for px in img_points:
        u, v = int(round(px[0])), int(round(px[1]))
        if 0 <= v < xyz_map.shape[0] and 0 <= u < xyz_map.shape[1]:
            pt_world = xyz_map[v, u]  # 注意顺序是[y, x]
            reprojected_world_points.append(pt_world)
        else:
            reprojected_world_points.append([np.nan, np.nan, np.nan])  # 越界处理

    reprojected_world_points = np.array(reprojected_world_points)
    dist_errors = np.linalg.norm(obj_points - reprojected_world_points, axis=1)
    mean_error = np.nanmean(dist_errors)

    return dist_errors, mean_error

In [25]:
# front
# 假设你已有内参 K 和畸变参数 D（4 个）
K = np.array([[511.88628231, 0, 967.12539385],
              [0, 510.43330913, 531.9395766],
              [0,  0,  1]])
D = np.array([0.12451778, -0.02741991, -0.00494669, 0.00162815])

# 世界坐标系中的棋盘格角点（Z=0）
obj_points = np.array([[0.8,0.8, 0],
                        [0.8,0.0, 0],
                       [0.8,-0.8, 0],
                       [1.6,-0.8, 0],
                        [1.6,0.0, 0],
                       [1.6,0.8, 0],
                       [2.4,0.8, 0],
                       [2.4,0.0, 0],
                    ], dtype=np.float32)

# 图像中的像素坐标（cv2.findChessboardCorners）
img_points = np.array([
    [554, 866],
    [979, 920],
    [1396, 854],
    [1244, 599],
    [972, 607],
    [698, 606],
    [778, 485],
    [969, 481]
], dtype=np.float32)

# 第一步：将畸变像素点转换为去畸变的归一化点
# 返回去畸变后的像素点
undistorted_pts = cv2.fisheye.undistortPoints(
    img_points.reshape(-1, 1, 2), K, D, P=K)
# reshape成 Nx2
undistorted_pts = undistorted_pts.reshape(-1, 2)
print("去畸变后的像素点：")
print(undistorted_pts)

# 第二步：调用普通solvePnP，传入去畸变后的像素点和无畸变的内参
retval, rvec, tvec = cv2.solvePnP(
    obj_points,
    undistorted_pts,
    K,
    distCoeffs=None,
    flags=cv2.SOLVEPNP_ITERATIVE
)
R_mat, _ = cv2.Rodrigues(rvec)
rotation = R.from_matrix(R_mat)
euler_angles = rotation.as_euler('ZYX', degrees=True)  # 输出 roll, pitch, yaw，单位是度
print(euler_angles)
# ✅ 验证：将 obj_points 投影回图像
# 注意：此时不带畸变，所以 distCoeffs=None
img_points_proj, _ = cv2.projectPoints(
    obj_points,
    rvec,
    tvec,
    K,
    distCoeffs=None
)
img_points_proj = img_points_proj.reshape(-1, 2)
print("投影点：")
print(img_points_proj)
# 计算重投影误差
errors = np.linalg.norm(undistorted_pts - img_points_proj, axis=1)
mean_error = np.mean(errors)

print("\n重投影误差 (单位：像素):")
for i, err in enumerate(errors):
    print(f"  点{i + 1}: 误差 = {err:.3f} px")
print(f"  平均误差 = {mean_error:.3f} px")

R_cam, _ = cv2.Rodrigues(rvec)
R_wc = R_cam.T
t_wc = -R_cam.T @ tvec

image_size = (1080, 1920)  # H, W

xyz_map = generate_pixel_to_world_map(K, D, R_wc, t_wc, image_size)

# 查看某像素的世界坐标
print("像素坐标对应的世界坐标:")
for px in img_points:
    u, v = int(round(px[0])), int(round(px[1]))
    print("Pixel (",v,",",u,") →", xyz_map[v, u])  # (X, Y, Z)
print("Pixel (554,866) →", xyz_map[606, 970])  # (X, Y, Z)
# save_xyz_map_binary(xyz_map, 'front_map.bin')

# 调用函数
dist_errors, mean_error = compute_world_projection_errors(img_points, obj_points, xyz_map)

# 打印结果
print("\n📏 3D 世界坐标误差 (单位：米):")
for i, err in enumerate(dist_errors):
    print(f"  点 {i+1}: 距离误差 = {err:.4f} m")
print(f"  平均距离误差 = {mean_error:.4f} m")

img_points = np.array([
    [1605, 695],
    [1454,545],
    [1359,468],
    [1250,395]

], dtype=np.float32)

# 世界坐标系中的棋盘格角点（Z=0）
obj_points = np.array([
    [1.06,-0.15-1.7485, 0],
    [1.90,-0.15-1.7485, 0],
    [2.61,-0.15-1.7485, 0],
    [3.85,-0.15-1.7485, 0],       
    ], dtype=np.float32)
# 调用函数
dist_errors, mean_error = compute_world_projection_errors(img_points, obj_points, xyz_map)

# 打印结果
print("\n📏 3D 世界坐标误差 (单位：米):")
for i, err in enumerate(dist_errors):
    print(f"  点 {i+1}: 距离误差 = {err:.4f} m")
print(f"  平均距离误差 = {mean_error:.4f} m")

去畸变后的像素点：
[[ 405.3136   986.2304 ]
 [ 980.8146   979.2999 ]
 [1554.9186   973.3388 ]
 [1264.2664   603.9086 ]
 [ 972.0223   607.3433 ]
 [ 679.1807   611.17883]
 [ 771.9366   483.49512]
 [ 969.0039   480.8935 ]]
[ -90.70172156  -58.62100138 -179.94599143]
投影点：
[[ 405.36115  986.2207 ]
 [ 980.43286  979.412  ]
 [1554.871    972.6107 ]
 [1265.409    604.6672 ]
 [ 972.0481   608.0361 ]
 [ 678.522    611.4068 ]
 [ 772.1215   482.97543]
 [ 969.174    480.73663]]

重投影误差 (单位：像素):
  点1: 误差 = 0.049 px
  点2: 误差 = 0.398 px
  点3: 误差 = 0.730 px
  点4: 误差 = 1.372 px
  点5: 误差 = 0.693 px
  点6: 误差 = 0.697 px
  点7: 误差 = 0.552 px
  点8: 误差 = 0.231 px
  平均误差 = 0.590 px
像素坐标对应的世界坐标:
Pixel ( 866 , 554 ) → [0.79998994 0.80005676 0.        ]
Pixel ( 920 , 979 ) → [ 8.0011827e-01 -5.3477520e-04  0.0000000e+00]
Pixel ( 854 , 1396 ) → [ 7.9919916e-01 -7.9927564e-01  1.1102230e-16]
Pixel ( 599 , 1244 ) → [ 1.6032627 -0.798521   0.       ]
Pixel ( 607 , 972 ) → [1.6029286e+00 2.7746790e-05 0.0000000e+00]
Pixel ( 606 

In [27]:

# back
# 假设你已有内参 K 和畸变参数 D（4 个）
K = np.array([[514.90744619, 0, 968.86462645],
              [0, 512.60283951, 546.76151604],
              [0,  0,  1]])
D = np.array([0.12347001, -0.021647, -0.00894293, 0.00232777])

# 世界坐标系中的棋盘格角点（Z=0）
obj_points = np.array([[-0.8,-0.8, 0],
                       [-0.8,0, 0],
                       [-1.6,-0.8, 0],
                       [-1.6,0.0, 0]
                    ], dtype=np.float32)

# 图像中的像素坐标（cv2.findChessboardCorners）
img_points = np.array([
    [582, 724],
    [953, 740],
    [713, 534],
    [953, 528],
], dtype=np.float32)

# 第一步：将畸变像素点转换为去畸变的归一化点
# 返回去畸变后的像素点
undistorted_pts = cv2.fisheye.undistortPoints(
    img_points.reshape(-1, 1, 2), K, D, P=K)

# reshape成 Nx2
undistorted_pts = undistorted_pts.reshape(-1, 2)
print("去畸变后的像素点：")
print(undistorted_pts)

# 第二步：调用普通solvePnP，传入去畸变后的像素点和无畸变的内参
retval, rvec, tvec = cv2.solvePnP(
    obj_points,
    undistorted_pts,
    K,
    distCoeffs=None,
    flags=cv2.SOLVEPNP_ITERATIVE
)
R_mat, _ = cv2.Rodrigues(rvec)
rotation = R.from_matrix(R_mat)
euler_angles = rotation.as_euler('ZYX', degrees=True)  # 输出 roll, pitch, yaw，单位是度
print(euler_angles)
# ✅ 验证：将 obj_points 投影回图像
# 注意：此时不带畸变，所以 distCoeffs=None
img_points_proj, _ = cv2.projectPoints(
    obj_points,
    rvec,
    tvec,
    K,
    distCoeffs=None
)
img_points_proj = img_points_proj.reshape(-1, 2)
print("投影点：")
print(img_points_proj)
# 计算重投影误差
errors = np.linalg.norm(undistorted_pts - img_points_proj, axis=1)
mean_error = np.mean(errors)

print("\n重投影误差 (单位：像素):")
for i, err in enumerate(errors):
    print(f"  点{i + 1}: 误差 = {err:.3f} px")
print(f"  平均误差 = {mean_error:.3f} px")

R_cam, _ = cv2.Rodrigues(rvec)
R_wc = R_cam.T
t_wc = -R_cam.T @ tvec

image_size = (1080, 1920)  # H, W

xyz_map = generate_pixel_to_world_map(K, D, R_wc, t_wc, image_size)

# 查看某像素的世界坐标
print("像素坐标对应的世界坐标:")
for px in img_points:
    u, v = int(round(px[0])), int(round(px[1]))
    print("Pixel (",v,",",u,") →", xyz_map[v, u])  # (X, Y, Z)
print("Pixel (554,866) →", xyz_map[606, 970])  # (X, Y, Z)

# save_xyz_map_binary(xyz_map, 'back_map.bin')

# 调用函数
dist_errors, mean_error = compute_world_projection_errors(img_points, obj_points, xyz_map)

# 打印结果
print("\n📏 3D 世界坐标误差 (单位：米):")
for i, err in enumerate(dist_errors):
    print(f"  点 {i+1}: 距离误差 = {err:.4f} m")
print(f"  平均距离误差 = {mean_error:.4f} m")


img_points = np.array([
    [251, 783],
    [412,602],
    [608,450],
    [683,403]

], dtype=np.float32)

# 世界坐标系中的棋盘格角点（Z=0）
obj_points = np.array([
    [-0.27,-0.15-1.7485, 0],
    [-1.12,-0.15-1.7485, 0],
    [-2.46,-0.15-1.7485, 0],
    [-3.27,-0.15-1.7485, 0],       
    ], dtype=np.float32)
# 调用函数
dist_errors, mean_error = compute_world_projection_errors(img_points, obj_points, xyz_map)

# 打印结果
print("\n📏 3D 世界坐标误差 (单位：米):")
for i, err in enumerate(dist_errors):
    print(f"  点 {i+1}: 距离误差 = {err:.4f} m")
print(f"  平均距离误差 = {mean_error:.4f} m")


去畸变后的像素点：
[[509.23978 757.3344 ]
 [952.4977  746.11865]
 [698.4136  533.27246]
 [952.9924  527.99097]]
[ 87.71058073  57.95399969 178.72111124]
投影点：
[[509.41895 757.31384]
 [952.25604 746.01245]
 [697.6941  533.3673 ]
 [953.8205  528.11414]]

重投影误差 (单位：像素):
  点1: 误差 = 0.180 px
  点2: 误差 = 0.264 px
  点3: 误差 = 0.726 px
  点4: 误差 = 0.837 px
  平均误差 = 0.502 px
像素坐标对应的世界坐标:
Pixel ( 724 , 582 ) → [-0.7999663 -0.8002958  0.       ]
Pixel ( 740 , 953 ) → [-7.9976034e-01  4.4235188e-04  0.0000000e+00]
Pixel ( 534 , 713 ) → [-1.6005036 -0.7979783  0.       ]
Pixel ( 528 , 953 ) → [-1.6008862  -0.00260613  0.        ]
Pixel (554,866) → [-1.2064927e+00  4.1781474e-02 -1.1102230e-16]

📏 3D 世界坐标误差 (单位：米):
  点 1: 距离误差 = 0.0003 m
  点 2: 距离误差 = 0.0005 m
  点 3: 距离误差 = 0.0021 m
  点 4: 距离误差 = 0.0028 m
  平均距离误差 = 0.0014 m

📏 3D 世界坐标误差 (单位：米):
  点 1: 距离误差 = 0.0936 m
  点 2: 距离误差 = 0.0840 m
  点 3: 距离误差 = 0.0956 m
  点 4: 距离误差 = 0.1355 m
  平均距离误差 = 0.1022 m


In [31]:
# left
# 假设你已有内参 K 和畸变参数 D（4 个）
K = np.array([[515.03734186, 0, 965.79609603],
              [0, 513.98728645, 539.65789101],
              [0,  0,  1]])
D = np.array([0.09528073, 0.06931891, -0.10041033, 0.03081875])

# 世界坐标系中的棋盘格角点（Z=0）
obj_points = np.array([
                        [0.8,1.6, 0],
                       [0.8,0.8, 0],
                       [1.6,0.0, 0],
                       [1.6,0.8, 0],
                       [0.0,0.8, 0],
                       [2.4,0.0, 0],
                       [2.4,0.8, 0],
                    ], dtype=np.float32)

# 图像中的像素坐标（cv2.findChessboardCorners）
img_points = np.array([
    [886, 558],
    [998, 749],
    [1475, 847],
    [1243, 640],
    [624, 863],
    [1552, 702],
    [1369, 571],
], dtype=np.float32)

# 第一步：将畸变像素点转换为去畸变的归一化点
# 返回去畸变后的像素点
undistorted_pts = cv2.fisheye.undistortPoints(
    img_points.reshape(-1, 1, 2), K, D, P=K)

# reshape成 Nx2
undistorted_pts = undistorted_pts.reshape(-1, 2)
print("去畸变后的像素点：")
print(undistorted_pts)

# 第二步：调用普通solvePnP，传入去畸变后的像素点和无畸变的内参
retval, rvec, tvec = cv2.solvePnP(
    obj_points,
    undistorted_pts,
    K,
    distCoeffs=None,
    flags=cv2.SOLVEPNP_ITERATIVE
)
R_mat, _ = cv2.Rodrigues(rvec)
rotation = R.from_matrix(R_mat)
euler_angles = rotation.as_euler('ZYX', degrees=True)  # 输出 roll, pitch, yaw，单位是度
print(euler_angles)
# ✅ 验证：将 obj_points 投影回图像
# 注意：此时不带畸变，所以 distCoeffs=None
img_points_proj, _ = cv2.projectPoints(
    obj_points,
    rvec,
    tvec,
    K,
    distCoeffs=None
)
img_points_proj = img_points_proj.reshape(-1, 2)
print("投影点：")
print(img_points_proj)
# 计算重投影误差
errors = np.linalg.norm(undistorted_pts - img_points_proj, axis=1)
mean_error = np.mean(errors)

print("\n重投影误差 (单位：像素):")
for i, err in enumerate(errors):
    print(f"  点{i + 1}: 误差 = {err:.3f} px")
print(f"  平均误差 = {mean_error:.3f} px")

R_cam, _ = cv2.Rodrigues(rvec)
R_wc = R_cam.T
t_wc = -R_cam.T @ tvec

image_size = (1080, 1920)  # H, W

xyz_map = generate_pixel_to_world_map(K, D, R_wc, t_wc, image_size)

# 查看某像素的世界坐标
print("像素坐标对应的世界坐标:")
for px in img_points:
    u, v = int(round(px[0])), int(round(px[1]))
    print("Pixel (",v,",",u,") →", xyz_map[v, u])  # (X, Y, Z)
print("Pixel (554,866) →", xyz_map[606, 970])  # (X, Y, Z)

# save_xyz_map_binary(xyz_map, 'left_map.bin')


# 调用函数
dist_errors, mean_error = compute_world_projection_errors(img_points, obj_points, xyz_map)

# 打印结果
print("\n📏 3D 世界坐标误差 (单位：米):")
for i, err in enumerate(dist_errors):
    print(f"  点 {i+1}: 距离误差 = {err:.4f} m")
print(f"  平均距离误差 = {mean_error:.4f} m")


img_points = np.array([
    [305, 586],
    [409,557],
    [577,516],
    [877,461]

], dtype=np.float32)

# 世界坐标系中的棋盘格角点（Z=0）
obj_points = np.array([
    [-2.03,0.15+2.2285, 0],
    [-1.31,0.15+2.2285, 0],
    [-0.46,0.15+2.2285, 0],
    [0.90,0.15+2.2285, 0],       
    ], dtype=np.float32)
# 调用函数
dist_errors, mean_error = compute_world_projection_errors(img_points, obj_points, xyz_map)

# 打印结果
print("\n📏 3D 世界坐标误差 (单位：米):")
for i, err in enumerate(dist_errors):
    print(f"  点 {i+1}: 距离误差 = {err:.4f} m")
print(f"  平均距离误差 = {mean_error:.4f} m")

去畸变后的像素点：
[[ 885.5199   558.11035]
 [ 999.31323  757.53674]
 [1734.2909  1003.50116]
 [1265.4193   648.11536]
 [ 540.1626   942.311  ]
 [1875.7859   791.6686 ]
 [1435.7734   576.1905 ]]
[-12.8508399  -21.30883377 124.91079201]
投影点：
[[ 885.9198   558.5509 ]
 [1000.00653  756.24615]
 [1732.7983  1001.2935 ]
 [1265.5726   648.23584]
 [ 539.0756   943.71484]
 [1876.681    792.5973 ]
 [1438.2617   578.0001 ]]

重投影误差 (单位：像素):
  点1: 误差 = 0.595 px
  点2: 误差 = 1.465 px
  点3: 误差 = 2.665 px
  点4: 误差 = 0.195 px
  点5: 误差 = 1.775 px
  点6: 误差 = 1.290 px
  点7: 误差 = 3.077 px
  平均误差 = 1.580 px
像素坐标对应的世界坐标:
Pixel ( 558 , 886 ) → [0.7995609 1.6029803 0.       ]
Pixel ( 749 , 998 ) → [7.9724163e-01 7.9789346e-01 1.1102230e-16]
Pixel ( 847 , 1475 ) → [ 1.5980365e+00 -1.8763008e-03 -1.1102230e-16]
Pixel ( 640 , 1243 ) → [1.6000154  0.80048597 0.        ]
Pixel ( 863 , 624 ) → [1.1195209e-03 8.0147511e-01 1.1102230e-16]
Pixel ( 702 , 1552 ) → [ 2.4019465e+00  1.3269631e-03 -1.1102230e-16]
Pixel ( 571 , 1369 ) 

In [ ]:

# right
# 假设你已有内参 K 和畸变参数 D（4 个）
K = np.array([[513.01296421, 0, 968.08212686],
              [0, 511.45945492, 529.89082084],
              [0,  0,  1]])
D = np.array([0.12957965, -0.02998568, -0.00449943, 0.00157089])

# 世界坐标系中的棋盘格角点（Z=0）
obj_points = np.array([[1.6,0.0, 0],
                       [2.4,0.0, 0],
                       [2.4,-0.8, 0],
                       [0.8,-0.8, 0],
                        [0.0,-0.8, 0],
                        [1.6,-0.8, 0]
                    ], dtype=np.float32)

# 图像中的像素坐标（cv2.findChessboardCorners）
img_points = np.array([
    [458, 844],
    [379, 700],
    [562, 572],
    [935, 746],
    [1304, 855],
    [689, 639],
], dtype=np.float32)

# 第一步：将畸变像素点转换为去畸变的归一化点
# 返回去畸变后的像素点
undistorted_pts = cv2.fisheye.undistortPoints(
    img_points.reshape(-1, 1, 2), K, D, P=K)

# reshape成 Nx2
undistorted_pts = undistorted_pts.reshape(-1, 2)
print("去畸变后的像素点：")
print(undistorted_pts)

# 第二步：调用普通solvePnP，传入去畸变后的像素点和无畸变的内参
retval, rvec, tvec = cv2.solvePnP(
    obj_points,
    undistorted_pts,
    K,
    distCoeffs=None,
    flags=cv2.SOLVEPNP_ITERATIVE
)
R_mat, _ = cv2.Rodrigues(rvec)
rotation = R.from_matrix(R_mat)
euler_angles = rotation.as_euler('ZYX', degrees=True)  # 输出 roll, pitch, yaw，单位是度
print(euler_angles)
# ✅ 验证：将 obj_points 投影回图像
# 注意：此时不带畸变，所以 distCoeffs=None
img_points_proj, _ = cv2.projectPoints(
    obj_points,
    rvec,
    tvec,
    K,
    distCoeffs=None
)
img_points_proj = img_points_proj.reshape(-1, 2)
print("投影点：")
print(img_points_proj)
# 计算重投影误差
errors = np.linalg.norm(undistorted_pts - img_points_proj, axis=1)
mean_error = np.mean(errors)

print("\n重投影误差 (单位：像素):")
for i, err in enumerate(errors):
    print(f"  点{i + 1}: 误差 = {err:.3f} px")
print(f"  平均误差 = {mean_error:.3f} px")

R_cam, _ = cv2.Rodrigues(rvec)
R_wc = R_cam.T
t_wc = -R_cam.T @ tvec

image_size = (1080, 1920)  # H, W

xyz_map = generate_pixel_to_world_map(K, D, R_wc, t_wc, image_size)


# save_xyz_map_binary(xyz_map, 'right_map.bin')

# 查看某像素的世界坐标
print("像素坐标对应的世界坐标:")
for px in img_points:
    u, v = int(round(px[0])), int(round(px[1]))
    print("Pixel (",v,",",u,") →", xyz_map[v, u])  # (X, Y, Z)
print("Pixel (554,866) →", xyz_map[606, 970])  # (X, Y, Z)

# 调用函数
dist_errors, mean_error = compute_world_projection_errors(img_points, obj_points, xyz_map)

# 打印结果
print("\n📏 3D 世界坐标误差 (单位：米):")
for i, err in enumerate(dist_errors):
    print(f"  点 {i+1}: 距离误差 = {err:.4f} m")
print(f"  平均距离误差 = {mean_error:.4f} m")

img_points = np.array([
    [741, 450],
    [847,471],
    [1019,505],
    [1378,585]

], dtype=np.float32)

# 世界坐标系中的棋盘格角点（Z=0）
obj_points = np.array([
    [2.61,-0.15-1.7485, 0],
    [1.90,-0.15-1.7485, 0],   
    [1.06,-0.15-1.7485, 0],
    [-0.27,-0.15-1.7485, 0],
    ], dtype=np.float32)
# 调用函数
dist_errors, mean_error = compute_world_projection_errors(img_points, obj_points, xyz_map)

# 打印结果
print("\n📏 3D 世界坐标误差 (单位：米):")
for i, err in enumerate(dist_errors):
    print(f"  点 {i+1}: 距离误差 = {err:.4f} m")
print(f"  平均距离误差 = {mean_error:.4f} m")

去畸变后的像素点：
[[ 191.36478  1008.1943  ]
 [  44.439472  796.611   ]
 [ 493.73297   579.07904 ]
 [ 933.6727    754.6706  ]
 [1385.2539    933.6394  ]
 [ 666.6596    647.73413 ]]
[-167.78864367  -20.86410481 -123.31795909]
投影点：
[[ 192.4321   1006.8443  ]
 [  43.620052  797.1382  ]
 [ 492.59308   579.5415  ]
 [ 932.3647    754.1277  ]
 [1386.1448    934.2752  ]
 [ 666.8625    648.7252  ]]

重投影误差 (单位：像素):
  点1: 误差 = 1.721 px
  点2: 误差 = 0.974 px
  点3: 误差 = 1.230 px
  点4: 误差 = 1.416 px
  点5: 误差 = 1.094 px
  点6: 误差 = 1.012 px
  平均误差 = 1.241 px
像素坐标对应的世界坐标:
Pixel ( 844 , 458 ) → [1.5990222e+00 1.2102699e-03 0.0000000e+00]
Pixel ( 700 , 379 ) → [ 2.4005103e+00 -9.8725199e-04  0.0000000e+00]
Pixel ( 572 , 562 ) → [ 2.3994336  -0.80297065  0.        ]
Pixel ( 746 , 935 ) → [ 0.79685366 -0.7999499   0.        ]
Pixel ( 855 , 1304 ) → [ 0.00107608 -0.8004437   0.        ]
Pixel ( 639 , 689 ) → [ 1.6037858 -0.802451   0.       ]
Pixel (554,866) → [ 0.9245772 -1.2795795  0.       ]

📏 3D 世界坐标误差 (单位：米):
 

In [ ]:
# 第二步：调用普通solvePnP，传入去畸变后的像素点和无畸变的内参
retval, rvec, tvec = cv2.solvePnP(
    obj_points,
    undistorted_pts,
    K,
    distCoeffs=None,
    flags=cv2.SOLVEPNP_ITERATIVE
)

print("rvec:", rvec)
print("tvec:", tvec)


In [ ]:
import cv2
import numpy as np
import cv2.aruco as aruco

# 定义字典（和你打印marker用的一致）
aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_6X6_100)

# 假设你有3个marker，ID分别是10, 20, 30
ids = np.array([10, 20, 30])

# marker边长（单位米）
marker_length = 0.04

# 手动定义每个marker四个角点的世界坐标，顺序对应OpenCV角点顺序:
# [top-left, top-right, bottom-right, bottom-left]
obj_points = []

# Marker 10，假设放置在原点，平面z=0
obj_points.append(np.array([
    [0, 0, 0],
    [marker_length, 0, 0],
    [marker_length, marker_length, 0],
    [0, marker_length, 0]
], dtype=np.float32))

# Marker 20，右移10cm放置
obj_points.append(np.array([
    [0.10, 0, 0],
    [0.10 + marker_length, 0, 0],
    [0.10 + marker_length, marker_length, 0],
    [0.10, marker_length, 0]
], dtype=np.float32))

# Marker 30，下移10cm放置
obj_points.append(np.array([
    [0, -0.10, 0],
    [marker_length, -0.10, 0],
    [marker_length, -0.10 + marker_length, 0],
    [0, -0.10 + marker_length, 0]
], dtype=np.float32))

# 创建自定义Board
board = aruco.Board_create(obj_points, aruco_dict, ids)

# 读取图像和相机内参示例（请替换）
img = cv2.imread('test_image.jpg')
K = ...  # 3x3相机矩阵
D = ...  # 畸变参数，鱼眼模型需自己调整

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 检测marker
corners, detected_ids, _ = aruco.detectMarkers(gray, aruco_dict)

if detected_ids is not None:
    # 估计自定义Board位姿
    retval, rvec, tvec = aruco.estimatePoseBoard(
        corners, detected_ids, board, K, D, None, None
    )
    if retval:
        print("Pose estimated successfully")
        R, _ = cv2.Rodrigues(rvec)
        T = np.eye(4)
        T[:3,:3] = R
        T[:3,3] = tvec.flatten()
        print("T_cam_to_board:\n", T)

        # 可视化
        aruco.drawDetectedMarkers(img, corners, detected_ids)
        aruco.drawAxis(img, K, D, rvec, tvec, 0.05)
        cv2.imshow('Pose', img)
        cv2.waitKey(0)
    else:
        print("Pose estimation failed")
else:
    print("No markers detected")


In [3]:
import cv2
import numpy as np
import glob

CHECKERBOARD = (6, 8)  # 角点数
square_size = 0.024  # 单位：米

# 构建世界坐标点（0,0,0), (1,0,0)..., (6,8,0)
objp = np.zeros((1, CHECKERBOARD[0]*CHECKERBOARD[1], 3), np.float32)
objp[0, :, :2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
objp *= square_size

objpoints = []  # 3D 点
imgpoints = []  # 2D 点

images = glob.glob("/home/ubuntu/Desktop/project/sensor-calibration/tmp/0702_office_952/back/*.jpg")

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD,
        cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)

    if ret:
        corners_subpix = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1),
            (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-6))

        objpoints.append(objp)
        imgpoints.append(corners_subpix)

# 标定
K = np.zeros((3, 3))
D = np.zeros((4, 1))
rvecs = []
tvecs = []

ret, K, D, rvecs, tvecs = cv2.fisheye.calibrate(
    objpoints, imgpoints, gray.shape[::-1], K, D, rvecs, tvecs,
    cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC  + cv2.fisheye.CALIB_FIX_SKEW,
    (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 1e-6)
)
# + cv2.fisheye.CALIB_CHECK_COND
print(f"OpenCV version: {cv2.__version__}")
print("K:\n", K)
print("D:\n", D)
print("RMS:", ret)


OpenCV version: 4.12.0
K:
 [[511.35258597   0.         967.25255915]
 [  0.         509.75262273 543.39529834]
 [  0.           0.           1.        ]]
D:
 [[ 0.12906492]
 [-0.02671141]
 [-0.00557208]
 [ 0.00148365]]
RMS: 122.55490454773349
